# Advanced Data Manipulation

Over the next two weeks or so, we'll develop a range of skills for advanced data analysis in Python. We'll handle questions like: 

1. What if our data is split over multiple files?
2. What if our data is in the wrong shape?
3. What if our data is too big to conveniently fit in memory?  

To approach these questions, we'll develop some more advanced skills with the familiar `pandas` library. We'll also introduce new tools for plotting, including spatial plots, and take a brief look at databases. 

## Case Study: Global Warming

Throughout our study, we'll focus on a single data set. This is the Global Historical Climatology Network data set, compiled by the National Centers for Environmental Information of the US National Oceanic and Atmospheric Administration. This data set contains an extraordinarily detailed record of surface temperatures at tens of thousands of atmospheric measurement stations across the globe. It is thus an ideal, publicly available data set for studying global warming. As you know, global warming is one of the most important issues of our time, primarily because it threatens the natural habitat of nature's most majestic creature: 

<figure class="image" style="width:50%">
  <img src="http://c.files.bbci.co.uk/CF3C/production/_111925035_penguino.jpg" alt="">
  <figcaption><i></i></figcaption>
</figure>

An additional reason to be concerned about global warming is that rising temperatures reduce snowfall, a necessary prerequisite for videos of panda bears falling off of snowmen: 

<figure class="image" style="width:50%">
  <img src="https://media1.tenor.com/images/c20400089a4f275b4c1b32ebe309d93f/tenor.gif" alt="">
  <figcaption><i></i></figcaption>
</figure>

### Takeaways for Today

Here's what I want you to take away from this lecture: 

1. Many **data visualization** problems can be solved by **data manipulation**. 
2. `pandas` provides a number of handy methods for combining and reshaping your data. 
3. Pandas are very clumsy animals. 


### Data Source and Format

I accessed the data [here](https://www.ncdc.noaa.gov/data-access/land-based-station-data/land-based-datasets/global-historical-climatology-network-monthly-version-4), and then did a small amount of manipulation in order to save it as a set of CSV files. The files are hosted online at 

> `https://raw.githubusercontent.com/PhilChodrow/PIC16B/master/datasets/noaa-ghcn/decades/`

There is a file for each decade between 1901 and 2020. Today, we'll use only the file containing the most recent data, 2011-2020. 

In [ ]:
import pandas as pd

# advanced plotting tools for data frames
# basically a bunch of matplotlib shortcuts
import seaborn as sns 

from matplotlib import pyplot as plt
import numpy as np

In [ ]:
interval = "2011-2020"
url = f"https://raw.githubusercontent.com/PhilChodrow/PIC16B/master/datasets/noaa-ghcn/decades/{interval}.csv"
df = pd.read_csv(url)

As provided, the data set contains the following columns: 

- `ID`: the ID number of the station. We can use this to figure out which country the station is in, as well as the spatial location of the station. 
- `Year`: the year of the measurement. 
- `VALUE1`-`VALUE12`: the temperature measurements themselves. `VALUE1` contains the temperature measurements for January, `VALUE2` for February, and so on. 
- The measurements are in hundredths of a degree, Celsius. 

We can now do things like plot the temperature in January for a selected station: 

At this station, the average temperature in January was typically between 20 and 22 degrees Celsius during the timeframe. 

While it can be useful to have the temperate time series, there's a lot of information missing from this plot. For example, we're only plotting for a single month of the year, and we are also only showing the observations from a single weather station. In this lecture, our goal will be to construct the following plot: 

<figure class="image" style="width:60%">
  <img src="https://raw.githubusercontent.com/PhilChodrow/PIC16B/master/lectures/EDA/pd-1-example-plot.png" alt="">
  <figcaption><i></i></figcaption>
</figure>

This is a plot of average temperatures over time for a selection of countries (ones whose names begin with E). In order to make this plot, there are two major data manipulation operations we need to perform. We need to:  

1. Figure out what *country* each station is in. 
2. Reshape the 12 month columns into a single long column. 

These two tasks correspond to *merging* and *stacking*. These operations will come up at key points in our workflow. 

### Adding Countries with Merges

A *merge* is an operation that combines two related data frames into a single data frame, using one or more columns as *keys*. Let's take a look at how this works. The first thing we'll do is acquire a data frame that gives the full country name corresponding to the FIPS code. The FIPS code is an internationally standardized abbreviation for a country:   

In [ ]:
countries_url = "https://raw.githubusercontent.com/mysociety/gaze/master/data/fips-10-4-to-iso-country-codes.csv"


How can we relate this to our `df` of temperature readings? Well, it turns out that the first two characters of the `ID` column give the FIPS code! 

Let's extract them with the `str` attribute. Note that I am creating a new column whose name matches exactly the corresponding column in the `countries` data frame. 

Now it's merge time. `pandas` provides a `merge()` function with many different arguments, as well as a related `DataFrame.join()` method. There are many different ways to merge two data sets, as explained in [this helpful chapter](https://jakevdp.github.io/PythonDataScienceHandbook/03.07-merge-and-join.html) of the Python Data Science Handbook.  The default for `pd.merge` is an *inner* merge. 

What's happened here is the following: 

1. If the FIPS code in a row of `df` matches a FIPS code in `countries`, then the corresponding columns of `countries` are added to that row in the `merge`d result. 
2. If the FIPS code in a row of `df` is not found in `countries`, then that row is no longer present in the `merge`d result. 

Other behavior is possible. For example, one might instead prefer that, in case 2, the corresponding parts of the row are populated with `NA` values. This corresponds to a *left* join (rather than the default *inner* join). 

We now have a few unnecessary columns, so we'll remove them. 

### Stacking

Recall that we are aiming to create this figure: 

<figure class="image" style="width:60%">
  <img src="https://raw.githubusercontent.com/PhilChodrow/PIC16B/master/lectures/EDA/pd-1-example-plot.png" alt="">
  <figcaption><i></i></figcaption>
</figure>

We now know the country of each observation station, so that's good progress! But we have a bit of a problem now: we'd like to be able to plot all the months in the same time series in the logical way. We can't do this right now because each month's data is in a separate column. How can we create a single column containing the temperature observations? 

The answer is that we can *stack* these columns into a single column. `pandas` provides a convenient method for `stack`ing and `unstack`ing data: 

<figure class="image" style="width:50%">
  <img src="https://media1.tenor.com/images/c20400089a4f275b4c1b32ebe309d93f/tenor.gif" alt="">
  <figcaption><i>Pandas unstacking.</i></figcaption>
</figure>



Here's how. First, we convert all the columns that we *don't* want to stack into a multi-index for the data frame. 

Then, we call the `stack()` method. This has the effect of "stacking" all of the data values on top of each other. There's also a new column indicating which of the original columns the observation came from: 

We can recover the `ID`, `Year`, and `Name` columns using `reset_index()`: 

This is looking pretty good, except that the final two columns aren't labeled very appropriately. Let's fix that up: 

Better! We are now very close to our goal. The final step is to create a `datetime` column that reflects both the year and month. First, we can extract the month by picking out everything after the `"VALUE"` in the `Month` column. 

Next, we'll create a new `datetime` column called `Date`. To do so, we can first create a string of the form `YYYY-MM`: 

We can convert the result to a `DateTime` using a built-in `pandas` function. The nice thing about this function is that it can automatically detect several common formats of date-time strings. 

### Plotting

We are finally ready to make our plot! First, we compute the average temperature across all stations in each month, and divide by 100 to get units of degrees Celsius. 

Next, we'll turn the index columns into regular columns for plotting purposes. 

Finally, we'll make the plot! To avoid overplotting, I'm going to plot only the countries whose English names begin with a given letter. The `lineplot` function of `seaborn` makes it easy to plot many labeled timeseries simultaneously. 

We did it! Constructing this plot from the supplied data required us to *merge* and *stack* our data. This is a common pattern in applied data analysis -- we need to manipulate our data in a number of ways prior to the cool plotting or machine learning. 

### Takeaways for Today

Here's what I want you to take away from this lecture: 

1. Many **data visualization** problems can be solved by **data manipulation**. 
2. `pandas` provides a number of handy methods for combining and reshaping your data. 
3. Pandas are very clumsy animals. 